# Time dependent vector spaces (work in progress)

This notebook describes the use of torchctrnn for situations in which the vector space (parameterised by a neural network) - a neural ODE - depends on both time $t$ and the encoded vector $h(t)$. 

The data model for the example is:

...

In [87]:
import torch
import torch.nn as nn

# torchctrnn library imports:
from torchctrnn import ODERNNCell

The cell below sets up our networks.

In [88]:
class ODENet(nn.Module):
    """"
    Subnetwork
    This neural network defines the vector space
    Time dependent
    """

    def __init__(self,hidden_size):
        super().__init__()

        self.hidden_size = hidden_size
        self.net = nn.Sequential(
            nn.Linear(1+hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
        )

    def forward(self,input, t, dt, hidden):
        z = torch.cat((t,hidden),1)
        return self.net(z)


class MainNet(nn.Module):
    """
    Full network
    """

    def __init__(self,ODENet,input_size,hidden_size,output_size):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.odenet = ODERNNCell(ODENet,input_size,hidden_size)
        self.decode = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, output_size),
            nn.Sigmoid(),
        )

    def forward(self,times,x):
        """
        Forward method for the main model

        x = (batch,timestep,features)
        times = 
        """

        J = x.size(1)
        batch_size = x.size(0)
        output = torch.zeros(batch_size,J,self.output_size)
        h = torch.zeros(batch_size,self.hidden_size)
        for j in range(0,J):
            xj = x[:,j,:]
            times_j = times[:,j]
            h = self.odenet(xj,h,times_j)
            output[:,j,:] = self.decode(h)
            h = h.squeeze(0)
        return output

In [89]:
x_size = 16  # number of input features
hidden_size = 4
output_size = 1

odenet = ODENet(hidden_size)
model = MainNet(odenet,x_size,hidden_size,output_size)

In [90]:
n_batch = 2
n_step = 10
x = torch.randn(n_batch,n_step,x_size) # (batch=1,timestep=1,features=10)
times = torch.zeros(n_batch,n_step,2)
times[:,:,1] = torch.rand(n_batch,n_step)

In [91]:
model(times,x)

tensor([[[0.4382],
         [0.4483],
         [0.3933],
         [0.4605],
         [0.4237],
         [0.3824],
         [0.4617],
         [0.4311],
         [0.4842],
         [0.4101]],

        [[0.4233],
         [0.3978],
         [0.4532],
         [0.4745],
         [0.4892],
         [0.4694],
         [0.4444],
         [0.4402],
         [0.4911],
         [0.4346]]], grad_fn=<CopySlices>)